# Proyecto Integrador Machine Learning por Francisco Garcia.

### Proceso de ETL:

Este proceso lo haremos relativamente rapido, ya que la presentacion del proyecto es en poco tiempo y el objetivo es deployar la API. Aun asi dejaremos la base lo mejor posible

Importamos librerias a utilizar.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('data\movies_and_credits.csv', index_col='id', low_memory=False)
data.head()

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,actors_names,_merge
id,,,,,,,,,,,,,,,,,,,,,
862,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",both
8844,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",both
15602,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Howard Deutch,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...",both
31357,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Forest Whitaker,"['Whitney Houston', 'Angela Bassett', 'Loretta...",both
11862,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Charles Shyer,"['Steve Martin', 'Diane Keaton', 'Martin Short...",both


Podemos Observar en primera instancia varias cosas.
- Hay bastantes faltantes
- Tenemos columnas con datos anidados.
- hay columnas que no nos sirven.

In [4]:
#Hacemos un info al dataset para una vista general.
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45542 entries, 862 to 461257
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             45542 non-null  int64  
 1   adult                  45542 non-null  object 
 2   belongs_to_collection  4503 non-null   object 
 3   budget                 45542 non-null  object 
 4   genres                 45542 non-null  object 
 5   homepage               7795 non-null   object 
 6   imdb_id                45525 non-null  object 
 7   original_language      45531 non-null  object 
 8   original_title         45542 non-null  object 
 9   overview               44588 non-null  object 
 10  popularity             45537 non-null  object 
 11  poster_path            45156 non-null  object 
 12  production_companies   45539 non-null  object 
 13  production_countries   45539 non-null  object 
 14  release_date           45455 non-null  object 
 15  reve

Con el uso de info() podemos observar que varias columnas no tienen el tipo de dato bien definido. Nos encargaremos de esto mas tarde.

In [5]:
#Eliminamos posibles duplicados.
df = data.drop_duplicates(subset=['title'])

Eliminamos las siguientes columnas que segun la documentacion no seran necesarias: ['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage']


In [6]:
cols_drop = ['video','Unnamed: 0', '_merge', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df = df.drop(cols_drop, axis=1)

Procedemos a chequear los valores faltantes en cada columna.

In [7]:
#lista de columnas actuales del df
columnas_df = ['belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average']

In [8]:
#lista de columnas con faltantes.
cols_with_missing = [column for column in columnas_df if df[column].isnull().sum() != 0]
cols_with_missing

['belongs_to_collection',
 'original_language',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average']

por el momento eliminaremos las filas con datos nulos en title, ya que no aportaran informacion relevante.

In [9]:
df.dropna(subset='title', inplace=True)

Eliminamos los faltantes y modificaremos el tipo de la columna "release_date" para poder trabajar correctamente con esta columna.

In [10]:
df.dropna(subset=['release_date'], inplace=True)
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year'] = df['release_date'].dt.year
df['release_year'].head()

id
862      1995
8844     1995
15602    1995
31357    1995
11862    1995
Name: release_year, dtype: int32

Observemos una vez mas info() para ver con que seguimos.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 862 to 461257
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4356 non-null   object        
 1   budget                 42196 non-null  object        
 2   genres                 42196 non-null  object        
 3   original_language      42185 non-null  object        
 4   overview               41294 non-null  object        
 5   popularity             42196 non-null  object        
 6   production_companies   42196 non-null  object        
 7   production_countries   42196 non-null  object        
 8   release_date           42196 non-null  datetime64[ns]
 9   revenue                42196 non-null  float64       
 10  runtime                41957 non-null  float64       
 11  spoken_languages       42196 non-null  object        
 12  status                 42121 non-null  object        
 13  tag

Ahora vamos a convertir los valores numericos a tipo float o int. podemos observar 2 columnas: 'budget', 'popularity'

In [12]:
df['budget'] = pd.to_numeric(df['budget'], downcast='float')
df['popularity'] = pd.to_numeric(df['popularity'], downcast='float')

Ahora que tenemos las columnas con el tipo que corresponde podemos crear una nueva columna llamada return con las columnas "budget" y "revenue"

In [24]:
df['return'] = (df['revenue'] - df['budget']) / df['budget']
df['return'].fillna(0, inplace=True)

In [25]:
df['return'].head(5)

id
862      11.451801
8844      3.043035
15602     0.000000
31357     4.090760
11862          inf
Name: return, dtype: float64

Tambien podemos aplicar un trim a todas las columnas de tipo objeto para evitar espacios innecesarios.

In [15]:
#columnas acutales del df
columnas_df2 = ['belongs_to_collection', 'budget', 'genres', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'release_year', 'return']

object_cols = [columna for columna in columnas_df if df[columna].dtype == 'O']

for i in object_cols:
    df[i].str.strip()

Por el momento todo va correctamente. ahora me gustaria desanidar las molestas columnas que tenemos. Para esto lo haremos en 2 partes:
- primero usaremos la columna "belongs_to_collection" ya que posee un string con un solo diccionario dentro.
- luego las columnas "production_companies", "production_countries", "genres", "spoken_languages", que tienen un formato de lista con varios diccionarios dentro.

In [16]:
#Primero rellenamos los datos faltantes con algo que podamos leer
df['belongs_to_collection'].fillna('N/D', inplace=True)

#Creamos una lista vacia para reunir los nombres
collection_names = []
#Creamos un bucle para iterar por el dataframe
for i in range(0, df.shape[0]):
    #inicializamos una variable que contendra cada fila.
    fila = df.iloc[i, 0]
    #si fila es nulo:
    if fila == 'N/D':
        #insertamos el valor nulo en el dataframe.
        collection_names.append(fila)
    else:
        #Evaluamos la fila para que lo tome como un diccionario.
        fila = eval(fila)
        #insertamos el valor de la llave key de nuestros diccionarios.
        collection_names.append(fila['name'])

#creamos un diccionario con los valores para luego hacer el dataframe.
collection_dict = {'belongs_to_collection': collection_names}
#obtenemos los indices de nuestro dataframe
df_indices = df.index
#lo convertimos en un dataframe
df_btc = pd.DataFrame(collection_dict, index=df_indices)
#cambiamos la columna actual del df por la nueva columna creada.
df['belongs_to_collection'] = df_btc
df.head(1)

,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,director,actors_names,release_year,return
id,,,,,,,,,,,,,,,,,,,,,
862,Toy Story Collection,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",1995,343554033.0


Podemos ver que con la columna "belongs_to_collection" quedo correctamente. para las proximas columnas usaremos una funcion.

Pero primero tenemos que limpiar estas 4 columnas.

In [17]:
#Imputamos los faltantes
df['genres'].fillna('N/D', inplace=True)
df['production_companies'].fillna('N/D', inplace=True)
df['production_countries'].fillna('N/D', inplace=True)
df['spoken_languages'].fillna('N/D', inplace=True)

#cambiamos corchetes vacios por N/D
df['genres'].replace('[]', 'N/D', inplace=True)
df['production_companies'].replace('[]', 'N/D', inplace=True)
df['production_countries'].replace('[]', 'N/D', inplace=True)
df['spoken_languages'].replace('[]', 'N/D', inplace=True)


Ahora si creamos la funcion.

In [18]:
def desanidar_columnas(indice_columna, nombre_columna):
    """
    Esta funcion se utiliza para desanidar una columna
    que contenga una lista con diccionarios y dentro de esos
    diccionarios la key: 'name'. devolviendo un dataframe con
    los nombres.

    parametros:
    -------
    indice_columna: el indice de la columna del dataframe a desanidar.
    nombre_columna: el nombre de la columna que contiene los nombres en
    el dataframe de salida.
    """
    #creamos una lista vacia donde guardaremos los nombres
    columnas_names = []
    #Iteramos sobre el df
    for i in range(0, df.shape[0]):
        #instanciamos la fila:
        fila = df.iloc[i, indice_columna]
        #si la fila es nulo
        if fila == 'N/D':
            #aniadimos la fila y paramos el loop
            columnas_names.append(fila)
        #sino
        else:
            #convertimos de str a list
            fila = eval(fila)
            #guardaremos los nombres de este indice en otra sublista
            fila_names = []
            #iteramos la lista
            for i in fila:
                #agregamos los nombres
                fila_names.append(i['name'])
            #agregamos la lista de nombres de la fila a nuestra lista oficial.
            columnas_names.append(fila_names)

    #creamos el diccionario con los datos
    diccionario = {nombre_columna: columnas_names}
    #creamos y devolvemos el dataframe
    dataframe = pd.DataFrame(diccionario, index=df_indices)
    return dataframe

Ahora que tenemos nuestra funcion se la aplicamos a las columnas

In [19]:
#creamos los dataframes por columna
df_gen = desanidar_columnas(2, 'genres')
df_pc = desanidar_columnas(6, 'production_companies')
df_count = desanidar_columnas(7, 'production_countries')
df_spk = desanidar_columnas(11, 'spoken_languages')

#los reemplazamos en el datafram
df.genres = df_gen
df.production_companies = df_pc
df.production_countries = df_count
df.spoken_languages = df_spk
df.head(1)

,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,spoken_languages,status,tagline,title,vote_average,vote_count,director,actors_names,release_year,return
id,,,,,,,,,,,,,,,,,,,,,
862,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,...,[English],Released,NaN,Toy Story,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",1995,343554033.0


Ahora todas nuestras columnas se ven correctamente. revisemos una vez mas info() para ver que nos falta.

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 862 to 461257
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  42196 non-null  object        
 1   budget                 42196 non-null  float64       
 2   genres                 42196 non-null  object        
 3   original_language      42185 non-null  object        
 4   overview               41294 non-null  object        
 5   popularity             42196 non-null  float32       
 6   production_companies   42196 non-null  object        
 7   production_countries   42196 non-null  object        
 8   release_date           42196 non-null  datetime64[ns]
 9   revenue                42196 non-null  float64       
 10  runtime                41957 non-null  float64       
 11  spoken_languages       42196 non-null  object        
 12  status                 42121 non-null  object        
 13  tag

Solo nos quedan 5 columnas con faltantes. los imputemos para ya dar por terminado el ETL.

In [21]:
#reemplazamos por el mas comun.
df['original_language'].fillna('en', inplace=True)
#reemplazamos sin data
df['overview'].fillna('N/D', inplace=True)
#reemplazamos por el promedio
df['runtime'].fillna(df['runtime'].mean(), inplace=True)
#reemplazamos por el mas comun.
df['status'].fillna('Released', inplace=True)
#reemplazamos sin data.
df['tagline'].fillna('N/D', inplace=True)

In [22]:
df.actors_names.replace('[]', 'N/D', inplace=True)
df.actors_names.fillna('N/D', inplace=True)
df.actors_names.info()

<class 'pandas.core.series.Series'>
Index: 42196 entries, 862 to 461257
Series name: actors_names
Non-Null Count  Dtype 
--------------  ----- 
42196 non-null  object
dtypes: object(1)
memory usage: 659.3+ KB


Ahora si podemos guardar nuestra base de datos tratada!

In [23]:
df.to_csv('data/peliculas_ETL.csv')